# GEOG 313 Final




Function 1: Read in Sentinel-1 SAR 

	- Inputs: AOI, date range 
	- Outputs: Median quarterly composites (mosaiced) of VH and VV bands 

Function 2: Create index time series 

	- Inputs: median quarterly composites, name of index (RVI, mRFDI) 
	- Outputs: time series of index value 

Function 3: Create Visualizations within subzones 

	- Inputs: time series, subzones 
	- outputs: graph of index value vs time for each subzone. 

Function 4: Visualize trends 

	- Inputs: time series, name of trend (linear, monotonic, etc) 
	- outputs: raster image of correlation coefficient or other values (depending on what trend is used- ex. for linear trend it would output two images, r value and correlation coefficient) 

Function 5: detect deforestation 

	- Inputs: time series, index threshold value 
	- outputs: image of what year deforestation occurred (when value decreased below the user-defined threshold. If the value decreased then increased the result will be inconclusive and marked with an appropriate value) 



In [1]:
import dask
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
print(client.dashboard_link)

http://127.0.0.1:8787/status


In [36]:
aoi = [38.5, -5.25, 38.75,-4.75]

start = "2024-01-01"
end = "2024-12-31"

bands = ["VV", "VH"]

In [37]:
search = get_sar_pc(aoi, start, end)
search



In [38]:

stack = stackstac.stack(
    search,
    epsg=32737,
    resolution=20,
    bounds_latlon=aoi,
    )

#stack.time.plot.hist(bins = 10)


In [42]:
q_composite = quartely_composite(stack)
q_composite

index = calculate_index(q_composite, "RVI")

# TO do- values within subzones
index.isel(time)plot.imshow()

<xarray.DataArray 'stackstac-b92a48a6a692cb62d4b458cb72296604' (time: 4,
                                                                y: 2766, x: 1388)> Size: 123MB
dask.array<truediv, shape=(4, 2766, 1388), dtype=float64, chunksize=(1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/33)
    band                                   <U2 8B 'vh'
  * x                                      (x) float64 11kB 4.445e+05 ... 4.7...
  * y                                      (y) float64 22kB 9.475e+06 ... 9.4...
    sar:pixel_spacing_range                int64 8B 10
    s1:instrument_configuration_ID         <U1 4B '7'
    sat:platform_international_designator  <U9 36B '2014-016A'
    ...                                     ...
    s1:resolution                          <U4 16B 'high'
    description                            <U173 692B 'Terrain-corrected gamm...
    raster:bands                           object 8B {'nodata': -32768, 'data...
    title                                  <U41 164B 'VH: vertical transmit, ...
    epsg                                   int64 8B 32737
  * time                                   (time) datetime64[ns] 32B 2024-03-...

In [ ]:
trends = linear_trend(index)
linecoeff = trends.sel(degree = 2)

linecoeff.polyfit_coefficients.plot.imshow()

In [28]:
import xarray as xr
import rioxarray as rxr
import numpy as np
import stackstac
import os
import leafmap
import requests
from tqdm import tqdm
import pystac_client
import planetary_computer
from matplotlib import pyplot as plt

# write function to get bbox for polygons


def get_sar_pc(bbox, start, end):

    catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",  # connect to Planetary Computer API
    modifier=planetary_computer.sign_inplace,               
    )

    search = catalog.search(filter_lang="cql2-json", filter={
        "op": "and",
        "args": [{"op": "s_intersects", "args": [{"property": "geometry"}, aoi]},
                 {"op": "=", "args": [{"property": "collection"}, "sentinel-1-rtc"]},
                 {"op": "anyinteracts", "args": [{"property": "datetime"}, f"{start}/{end}"]},
                 {"op": "=", "args": [{"property": "sar:polarizations"}, ["VV", "VH"]]}
                ]
    }
                           )

    items = search.get_all_items()  # get items into new variable
    return items


def items_to_xarray(items, bands):
    stack = stackstac.stack(
        items,                 # output of item collection search
   #     bounds_latlon=bbox,       # clip to bbox
        assets=bands           # get only green and nir bands
    )
    
    return stack

def quartely_composite(stack):
    composite = stack.resample(time="1Q").median("time")

    return composite

def calculate_index(stack, index):
    vh, vv = stack.sel(band = "vh"), stack.sel(band = "vv")

    if index == "mRFDI":
        value = (vv-vh)/(vv+vh)
        return value
    elif index == "RVI":
        value = (4*vh)/(vv+vh)
        return value
    else:
        print("Invalid index name- please choose mRFDI, or RVI")

def linear_trend(stack):
    trend = stack.polyfit(dim = "time", deg = 1)
    return trend

    